# Google Calendar

## Introductionn

## Setup

### Imports

In [ ]:
import sys
sys.path.append('../../src/')

from utils.constants import SUPPORTED_PLATFORMS, SUPPORTED_SONGS

import pandas as pd
import numpy as np

from datetime import datetime, timezone
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

### Constants

In [ ]:
KONTRAS_CALENDAR_ID = 'jm4irm09dtur36oski2kighdug@group.calendar.google.com'

### Create Client

In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

## Query Calendar

In [ ]:
def clean_datetime(dt):
    if type(dt) != float:
        return dt.replace('T',' ').replace('+02:00','')
    else: 
        return dt

In [ ]:
def get_events(calendar_id ,start_date, end_date):
    """
    Function that gets all the events for calendar_id for the date range.
    
    Parameters
    calendar_id(str): Google Calendar ID
    start_date(str): Start date in the format '2019-01-20'
    end_date(str): End date in the format '2019-01-20'
    """
    
    s_year, s_month, s_day = start_date.split('-')
    e_year, e_month, e_day = end_date.split('-')
    
    s_date = datetime(int(s_year), int(s_month), int(s_day), 
                      0, 0, 0, 0).isoformat() + 'Z'
    e_date = datetime(int(e_year), int(e_month), int(e_day), 
                      23, 59, 59, 0).isoformat() + 'Z'
    
    events_result = service.events().\
    list(calendarId=KONTRAS_CALENDAR_ID, 
         timeMin=s_date,
         timeMax=e_date,
         maxResults=2500, 
         singleEvents=True,
         orderBy='startTime').execute()

    df = pd.DataFrame(events_result.get('items', []))
    
    df['start'] = pd.to_datetime(df.start.\
                             apply(lambda x: clean_datetime(x.get('dateTime', 
                                                                  np.NAN))), 
              format='%Y-%m-%d %H:%M:%S')
    df['end'] = pd.to_datetime(df.end.\
                                 apply(lambda x: clean_datetime(x.get('dateTime', 
                                                                      np.NAN))), 
                  format='%Y-%m-%d %H:%M:%S')
    
    return df

In [ ]:
df = get_events(KONTRAS_CALENDAR_ID, '2019-08-01', '2020-01-05')

In [ ]:
df.start.min(), df.start.max()

In [ ]:
df.summary.unique()

## Define Codes

Calendar entry structure:

```
Release 24

@KONTRAS_EVENT
PLATFORM: Spotify
GCAL_CODE: SONG_RELEASE
SONG: 24
MESSAGE: Releasing 24
VALUE: 
```

Where:

+ `@KONTRAS_EVENT` is an indicator that the event needs to be parsed by the bot
+ `PLATFORM`, as defined in SUPPORTED_PLATFORMS
+ `GCAL_CODE`, we'll define these now
+ `SONG`, as defined in SUPPORTED_SONGS
+ `MESSAGE`, any string
+ `VALUE`, any string

### GCAL_CODES

`GCAL_CODES` for now.

+ MARKETING_CAMPAIGN
+ SONG_RELEASE
+ GIG
+ SONG_RECORD
+ SOCIAL_MEDIA

In [ ]:
SUPPORTED_GCAL_CODES = ['MARKETING_CAMPAIGN',
                        'SONG_RELEASE',
                        'GIG',
                        'SONG_RECORD',
                        'SOCIAL_MEDIA']

In [ ]:
df = get_events(KONTRAS_CALENDAR_ID, 
                start_date='2020-01-17', 
                end_date='2020-03-19')

In [ ]:
def filter_valid_events(summary):
    return '@KONTRAS_EVENT' in summary

In [ ]:
def parse_summary(summary):
    payload = summary.split('@KONTRAS_EVENT')[1].strip()
    items = payload.split('\n')
    result = {}
    for item in items:
        k, v = [i.strip() for i in item.split(':')]
        result[k] = v
    return result

In [ ]:
valid_events_df = df[df.summary.apply(lambda x: filter_valid_events(x))]
valid_events_df.summary.apply(lambda x: parse_summary(x))

In [ ]:
valid_events_df

## Drawing Campaign Plots

In [ ]:
import plotly.express as px

In [ ]:
valid_events_df['dummy_val'] = 10

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for row in valid_events_df.iterrows():
    _ = row[1]
    start = _.start
    end = _.end
    details = parse_summary(_.summary)
    
    gcal_code = details.get('GCAL_CODE','NO_GCAL_CODE')
    message = details.get('MESSAGE','NO_MESSAGE')
    value = details.get('VALUE','NO_VALUE')
    platform = details.get('PLATFORM','NO_PLATFORM')
    song = details.get('SONG','NO_SONG')
    
    hover_text = message
    
    fig.add_trace(go.Scatter(x=[start, end], 
                             y=[2, 2],
                             hoverinfo='text',
                             hovertext=hover_text,
                             fill='tozeroy')) # fill down to xaxis

fig.update_layout(template='plotly_dark')
fig.show()